We employ Sentece-CamemBERT to enhance the performance of our natural language processing task. To further boost the model's proficiency, we augment our dataset. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [2]:
# Read the CSV file
training_data = pd.read_csv("../Dataset_upgrade/training_dataUP.csv", index_col=0)
#training_data = training_data.drop(columns=['note_orthographe'])
training_data.head()

,sentence,difficulty,note_orthographe,lexical_complexite,char_length,word_length,type_token_ratio,sentence_length,avg_word_length,complexite_texte,...,DET,PRON,NUM,NOUN,INTJ,ADP,ADJ,VERB,PROPN,SCONJ
id,,,,,,,,,,,,,,,,,,,,,
0,Les coûts kilométriques réels peuvent diverger...,4,1.000000,0.194007,0.160077,0.140152,0.467105,0.140152,0.339713,0.244565,...,0.066667,0.000000,0.000000,0.311111,0.0,0.288889,0.066667,0.088889,0.0000,0.0
1,"Le bleu, c'est ma couleur préférée mais je n'a...",0,1.000000,0.082334,0.036990,0.041667,1.000000,0.041667,0.204545,0.086957,...,0.187500,0.125000,0.000000,0.125000,0.0,0.000000,0.000000,0.125000,0.0625,0.0
2,Le test de niveau en français est sur le site ...,0,0.769231,0.088078,0.039541,0.045455,0.826923,0.045455,0.195804,0.081522,...,0.200000,0.000000,0.000000,0.400000,0.0,0.266667,0.000000,0.066667,0.0000,0.0
3,Est-ce que ton mari est aussi de Boston?,0,1.000000,0.062664,0.022959,0.026515,1.000000,0.026515,0.193182,0.054348,...,0.000000,0.100000,0.000000,0.300000,0.0,0.100000,0.000000,0.100000,0.1000,0.1
4,"Dans les écoles de commerce, dans les couloirs...",2,1.000000,0.184993,0.130740,0.125000,0.602941,0.125000,0.288770,0.228261,...,0.095238,0.047619,0.047619,0.238095,0.0,0.261905,0.047619,0.095238,0.0000,0.0


**pre-trained model BERT (large) in french**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

# pre-trained model
model_name = "dangvantuan/sentence-camembert-large"
model = SentenceTransformer(model_name)

phrases = training_data['sentence'].tolist()

# embeddings
bert_embeddings = model.encode(phrases)

No sentence-transformers model found with name /Users/phil/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [4]:
embedding_size = bert_embeddings.shape[0]
print("Embedding size:", embedding_size)

Embedding size: 4800


train test split

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

training_data_x = training_data.drop(columns=["difficulty", "sentence"])
X_combined = np.concatenate((bert_embeddings, training_data_x), axis=1)

X = X_combined  # embeddings
y = training_data["difficulty"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**LgisticRegression (best accuracy) :**

In [6]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()

# Train
logistic_regression_model.fit(X_train, y_train)

# Predict on the test set
y_pred = logistic_regression_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Logistic Regression model:", accuracy)

# Delete the model to free up memory
del logistic_regression_model

Accuracy of the Logistic Regression model: 0.4947916666666667


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM (Support Vector Machine) :
Accuracy with the best parameters

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Définir les paramètres pour la recherche par grille
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'] 
}

# model SVM + GridSearchCV
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', verbose=2)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ", grid_search.best_params_)

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Précision avec les meilleurs paramètres : ", accuracy)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   4.7s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   4.7s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   4.7s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   4.6s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   4.6s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   8.3s


KeyboardInterrupt: 

In [9]:
from sklearn.svm import SVC

# modèle SVM (classification)
svm_model = SVC(C=10, gamma='scale', kernel='rbf')

# train
svm_model.fit(X_train, y_train)


# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the SVM model:", accuracy)

# Delete the model to free up memory
del svm_model

Accuracy of the SVM model: 0.5520833333333334


Réseaux de Neurones (utilisant scikit-learn) : Accuracy with the best parameters

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Taille des vecteurs d'embeddings de CamemBERT
input_shape = 1047 

# Construction du modèle
model = models.Sequential([
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(input_shape,)),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(y.nunique(), activation='softmax')
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

# Évaluation du modèle
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Précision sur l'ensemble de test : ", test_accuracy)


Epoch 1/20


2023-12-04 15:56:31.512976: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/keras/engine/input_spec.

Réseaux de Neurones Récursifs - RNN

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# Paramètres à tester
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],  
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp = MLPClassifier(max_iter=1000)

# GridSearchCV
grid_search = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Affichez les meilleurs paramètres
print("Meilleurs paramètres:", grid_search.best_params_)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

Meilleurs paramètres: {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'sgd'}


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# CMLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=5000, random_state=42, alpha=0.05, learning_rate='constant', solver='sgd')

# Train
mlp_model.fit(X_train, y_train)

y_pred_mlp = mlp_model.predict(X_test)

# Accuracy
accuracy_mlp = mlp_model.score(X_test, y_test)
print("Précision du modèle MLPClassifier :", accuracy_mlp)

print(classification_report(y_test, y_pred_mlp))

del mlp_model

Précision du modèle MLPClassifier : 0.5229166666666667
              precision    recall  f1-score   support

           0       0.62      0.64      0.63       166
           1       0.40      0.42      0.41       158
           2       0.48      0.45      0.46       166
           3       0.49      0.52      0.50       153
           4       0.48      0.49      0.49       152
           5       0.66      0.61      0.64       165

    accuracy                           0.52       960
   macro avg       0.52      0.52      0.52       960
weighted avg       0.52      0.52      0.52       960

